In [48]:
import requests
import json
import pandas as pd
import time
from dateutil.parser import parse
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

apikey = 'X1o3cmgN7nsBDsInu4km3XtlPD051oPC'
wordType = 'matche'
main_url = 'https://api.neople.co.kr/cy/'
player_url = 'https://api.neople.co.kr/cy/players/'
matches_url = 'https://api.neople.co.kr/cy/matches/'
pd.options.display.max_columns = 300
columnsname = ['nickname','characterName','position','attribute1','attribute2','attribute3','hand','hand(rarityName)','head','head(rarityName)','chest','chest(rarityName)','waist','waist(rarityName)','leg','leg(rarityName)','foot','foot(rarityName)','accessories1','accessories1(rarityName)','accessories2','accessories2(rarityName)','recovery','recovery(rarityName)','attack','attack(rarityName)','accelerator','accelerator(rarityName)','defense','defense(rarityName)','special','special(rarityName)','neck','neck(rarityName)','accessories3','accessories3(rarityName)','accessories4','accessories4(rarityName)','random','level','killCount','deathCount','assistCount','attackPoint','damagePoint','battlePoint','sightPoint','getCoin','spendConsumablesCoin','sentinelKillCount','demolisherKillCount','playTime','minLifeTime','maxLifeTime','responseTime','team','matchId','map']
basic = pd.DataFrame(columns=columnsname)


In [49]:
# 데이터 수집단 함수 정리
def get_playerid(nickname):#플레이어 고유 id를 구하는 함수
    api = main_url+'players?nickname='+nickname+'&wordType='+wordType+'&apikey='+apikey
    playerdata = requests.get(api)
    playerId = playerdata.json()['rows'][0]['playerId']
    return playerId

#매치 id 가져오기
def get_matches(playerId):
    matchids = []
    gameTypeId = 'rating'
    #검색 시작일
    startDate='2022-01-01'
    endDate='2022-03-30'
    limit = 1000
    player_url = 'https://api.neople.co.kr/cy/players/'
    cc = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate
    playdata = requests.get(cc)
    playdata_info= playdata.json()
    while True:
        try:
            for matche in playdata_info['matches']['rows']:
                matchids.append(matche['matchId'])
            next = playdata_info['matches']['next']
            cc_next = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate+'&next='+next    
            playdata = requests.get(cc_next)
            playdata_info= playdata.json()
            time.sleep(0.5)
        except:
            break
        
    return matchids


def win_and_lose(matchId):#매치 id를 들고와서 승리팀 조합과,패배팀 조합으로 나눠줌
    api3=matches_url+matchId+'?&apikey='+apikey
    game = requests.get(api3)
    lose_team = []
    win_team = []
    lose_ch = []
    win_ch = []
    try:
        if game.json()['teams'][0]['result'] == 'lose':
            for lose_player in game.json()['teams'][0]['players']:
                lose_team.append(lose_player)
            for win_player in game.json()['teams'][1]['players']:
                win_team.append(win_player)
        elif game.json()['teams'][0]['result'] == 'win':
            for lose_player in game.json()['teams'][1]['players']:
                lose_team.append(lose_player)
            for win_player in game.json()['teams'][0]['players']:
                win_team.append(win_player)
        lose_ch =[]
        win_ch = []
        for i in range(0,10):
            if game.json()['players'][i]['playerId'] in lose_team:
                lose_ch.append(game.json()['players'][i]['playInfo']['characterName'])
            elif game.json()['players'][i]['playerId'] in win_team:
                win_ch.append(game.json()['players'][i]['playInfo']['characterName'])
    except:pass
    return win_ch,lose_ch

#중복매칭 데이터 방지를 위한 데이터 불러오기
def road_antdpData():
    ant_data = []
    antdpData = pd.read_csv('MatcesData/antdpData.csv')
    antdpData.drop('Unnamed: 0',axis=1,inplace=True)
    #csv를 리스트로 변환
    for v in antdpData['0']:
        ant_data.append(v)
    return ant_data

# 이번에 얻은 데이터를 중복방지 데이터에 기록하며 저장
def antdpData_save(antdpData,now_matcs):
    antdpData.extend(now_matcs)
    antdpData = pd.DataFrame(antdpData)
    antdpData.to_csv('MatcesData/antdpData.csv')
    
    


In [50]:
#데이터 수집 상세단
def ingame_data(matchId):
    api3=matches_url+matchId+'?&apikey='+apikey
    game = requests.get(api3).json()
    basic = pd.DataFrame(columns=columnsname)
    win_ch,lose_ch = win_and_lose(matchId)
    for i in range(0,10):
        ch = []
        if game['players'][i]['playInfo']['characterName'] in lose_ch:
            ch.append(game['players'][i]['nickname'])
            ch.append(game['players'][i]['playInfo']['characterName'])
            ch.append(game['players'][i]['position']['name'])
            for j in range(0,3):
                if j == 0:
                    ch.append(game['players'][i]['position']['attribute'][j]['name'])
                if j == 1:
                    ch.append(game['players'][i]['position']['attribute'][j]['name'])
                if j == 2:
                    ch.append(game['players'][i]['position']['attribute'][j]['name'])
            for k in range(0,16):
                ch.append(game['players'][i]['items'][k]['itemName'])
                ch.append(game['players'][i]['items'][k]['rarityName'])
            ch.append(game['players'][i]['playInfo']['random']) # 랜덤 유무
            ch.append(game['players'][i]['playInfo']['level'])#레벨
            ch.append(game['players'][i]['playInfo']['killCount'])#죽인 횟수
            ch.append(game['players'][i]['playInfo']['deathCount'])#죽은 횟수
            ch.append(game['players'][i]['playInfo']['assistCount'])#어시
            ch.append(game['players'][i]['playInfo']['attackPoint'])#준 데미지
            ch.append(game['players'][i]['playInfo']['damagePoint'])#받은 데미지
            ch.append(game['players'][i]['playInfo']['battlePoint'])#전투점수
            ch.append(game['players'][i]['playInfo']['sightPoint'])#시야점수
            ch.append(game['players'][i]['playInfo']['getCoin'])#흭득 코인
            ch.append(game['players'][i]['playInfo']['spendConsumablesCoin']) # 소모품 코인
            ch.append(game['players'][i]['playInfo']['sentinelKillCount']) # 센티널(중립) 코인
            ch.append(game['players'][i]['playInfo']['demolisherKillCount']) # 철거반 코인
            ch.append(game['players'][i]['playInfo']['playTime'])
            ch.append(game['players'][i]['playInfo']['minLifeTime'])
            ch.append(game['players'][i]['playInfo']['maxLifeTime'])
            ch.append(game['players'][i]['playInfo']['responseTime'])
            ch.append(0)
            ch.append(matchId)
            ch.append(game['map']['name'])
            ch = pd.DataFrame(ch)
            ch = ch.transpose()
            ch.columns = columnsname
            basic = pd.concat([basic,ch])
    for i in range(0,10):
        ch = []
        if game['players'][i]['playInfo']['characterName'] in win_ch:
            ch.append(game['players'][i]['nickname'])
            ch.append(game['players'][i]['playInfo']['characterName'])
            ch.append(game['players'][i]['position']['name'])
            for j in range(0,3):
                if j == 0:
                    ch.append(game['players'][i]['position']['attribute'][j]['name'])
                if j == 1:
                    ch.append(game['players'][i]['position']['attribute'][j]['name'])
                if j == 2:
                    ch.append(game['players'][i]['position']['attribute'][j]['name'])
            for k in range(0,16):
                ch.append(game['players'][i]['items'][k]['itemName'])
                ch.append(game['players'][i]['items'][k]['rarityName'])
            ch.append(game['players'][i]['playInfo']['random']) # 랜덤 유무
            ch.append(game['players'][i]['playInfo']['level'])#레벨
            ch.append(game['players'][i]['playInfo']['killCount'])#죽인 횟수
            ch.append(game['players'][i]['playInfo']['deathCount'])#죽은 횟수
            ch.append(game['players'][i]['playInfo']['assistCount'])#어시
            ch.append(game['players'][i]['playInfo']['attackPoint'])#준 데미지
            ch.append(game['players'][i]['playInfo']['damagePoint'])#받은 데미지
            ch.append(game['players'][i]['playInfo']['battlePoint'])#전투점수
            ch.append(game['players'][i]['playInfo']['sightPoint'])#시야점수
            ch.append(game['players'][i]['playInfo']['getCoin'])#흭득 코인
            ch.append(game['players'][i]['playInfo']['spendConsumablesCoin']) # 소모품 코인
            ch.append(game['players'][i]['playInfo']['sentinelKillCount']) # 센티널(중립) 코인
            ch.append(game['players'][i]['playInfo']['demolisherKillCount']) # 철거반 코인
            ch.append(game['players'][i]['playInfo']['playTime'])
            ch.append(game['players'][i]['playInfo']['minLifeTime'])
            ch.append(game['players'][i]['playInfo']['maxLifeTime'])
            ch.append(game['players'][i]['playInfo']['responseTime'])
            ch.append(0)
            ch.append(matchId)
            ch.append(game['map']['name'])
            ch = pd.DataFrame(ch)
            ch = ch.transpose()
            ch.columns = columnsname
            basic = pd.concat([basic,ch])
    return basic

In [51]:
matchId = 'faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec6db2885f593b26a552'
b = ingame_data(matchId)
basic = pd.concat([basic,b])

In [54]:
basic = basic.reset_index(drop=True)
basic.to_csv('test.csv',index=False, encoding="utf-8-sig")

In [42]:
basic

,nickname,characterName,position,attribute1,attribute2,attribute3,hand,hand(rarityName),head,head(rarityName),chest,chest(rarityName),waist,waist(rarityName),leg,leg(rarityName),foot,foot(rarityName),accessories1,accessories1(rarityName),accessories2,accessories2(rarityName),recovery,recovery(rarityName),attack,attack(rarityName),accelerator,accelerator(rarityName),defense,defense(rarityName),special,special(rarityName),neck,neck(rarityName),accessories3,accessories3(rarityName),accessories4,accessories4(rarityName),random,level,killCount,deathCount,assistCount,attackPoint,damagePoint,battlePoint,sightPoint,getCoin,spendConsumablesCoin,sentinelKillCount,demolisherKillCount,playTime,minLifeTime,maxLifeTime,responseTime,team,matchId
0,뉘들이닭맛을알아,나이오비,원거리딜러,회피 사격,돌격전차,완전한 성장,E 레이지 파이어월,유니크,E 앨더란,유니크,E 버밀리온 하트,유니크,S 플레임 시타델,유니크,E 그레이스 스콧,유니크,E 불꽃의 윤무,유니크,불놀이 로임,레어,불벽 미루브렛,레어,루비 스파클링,레어,런업 스프린터,레어,블레이즈 파이크,레어,솔리드 스위퍼,레어,불놀이 임팩트,언커먼,S 인퍼널 배일,유니크,정념폭발 레그람,레어,헬 세티스피아,유니크,False,47,2,5,8,22218,15013,114,142,14270,2900,28,15,919,48,431,99,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
1,악당공룡,드렉슬러,원거리딜러,완벽주의자,사냥꾼의 본능,완전한 성장,E 마노 레파르토,유니크,E 린 아르메,유니크,E 드래곤 스카,유니크,E 플랑베르 라비타,유니크,E 캐러비니어,유니크,E 폴링 스타,유니크,심판 시트리,레어,분열의 시트리,레어,스피어 스파클링,레어,런업 스프린터,레어,파이크 이펙션,레어,가디언 타즈,레어,분열창 임팩트,언커먼,E 프라디지 드라군,유니크,나선창 시트리,레어,유성 무차스랜서,유니크,False,54,2,6,8,24334,18941,93,127,17070,2900,18,31,919,48,254,147,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
2,티안나게잘던짐,케니스,탱커,최후의 저항,재생의 숨결,불굴의 의지,E 격변의 시대,유니크,S 완벽한 후계자,유니크,S 비질란테,유니크,S 이상적인 리더,유니크,E 꿈꾸는 자,유니크,E 인헤리터,유니크,스파우트 레그람,레어,피크 레그람,레어,체인지 스파클링,레어,체인지 스프린터,레어,파이크 이펙션,레어,체인지 타즈,레어,PS 레이더 R4,언커먼,S 하츠 앵커,유니크,SB래피드 데브,레어,자유의 갈망,유니크,False,38,0,9,3,4947,49495,142,189,10990,1900,7,15,919,31,169,237,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
3,이론개론,제레온,탱커,전투 태세,재생의 숨결,최후의 저항,FA제이더 건틀릿,레어,E 루미넌 애퀼러,유니크,S 로열 히페리온,유니크,E 템플러 소린,유니크,E 헤본 가디언,유니크,S 인그레이 가디언,유니크,가디언 도미닌,레어,프렐류드 레그람,레어,첼시 콜라 V,레어,리세스 스프린터,레어,로열 파이크,레어,로열 타즈,레어,템페스트 임팩트,언커먼,E 바요넷 플로리쉬,유니크,템페스트 도미닌,레어,레퀴엠 도미닌,레어,False,39,1,9,3,12306,45217,186,214,10890,2000,7,2,919,21,314,202,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
4,리나,시바,근거리딜러,급소 가격,할인 판매,거인 사냥꾼,SP다커 댄 블랙,레어,E 페르소나,유니크,하트 브레이커,레어,E 다크 어비스,유니크,E 블랙 나이트메어,유니크,S 킬링 댄스,유니크,킬힐 비프론즈,레어,절개의 비프론즈,레어,다크 스파클링,레어,리세스 스프린터,레어,데블 파이크,레어,스테민 터틀,레어,절개 임팩트,언커먼,레드 나이트메어,레어,SB칼날힐 데브,레어,데빌런-섀도우,레어,False,62,11,4,3,48091,14113,103,131,21210,4260,42,48,919,76,517,149,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
5,윤두부,레나,탱커,기민한 몸놀림,충동 구매,최후의 저항,FA샤프 블레이저,레어,E 사이퍼 스카우터,유니크,S 랜드 스피더,유니크,RA베스핀 트론,레어,E 글렘스타 라이트,유니크,S 펌프 나이트로,유니크,축퇴 레그람,레어,금강 헤르메스,레어,볼륨업 스파클링,레어,엑셀레이션,레어,이펙트 이펙션,레어,기어 플래쉬,레어,PS 레이더 R4,언커먼,블루 래빗,레어,SB섬전각 데브,레어,기어2 레그람,레어,False,40,3,4,13,13101,32091,159,245,13140,3600,15,6,919,62,344,91,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
6,방구석게임폐인,윌라드,원거리딜러,충동 구매,할인 판매,완전한 성장,프라디 보크스,레어,E 그랜드 듀크,유니크,S 블레스트 슈트,유니크,E 청천벽력,유니크,S 스틸레토라,유니크,S 라이트 블리츠,유니크,낙뢰 오리악스,레어,구형 루드크리스,레어,더스팅 스파클링,레어,엑셀레이션,레어,라이트닝 이펙트,레어,솔리드 스위퍼,레어,낙뢰 임팩트,언커먼,E 블리츠 와일즈,유니크,DB전기 베티,레어,썬더 고르드마스,유니크,False,61,8,4,13,37081,23255,146,150,18640,2360,26,21,919,46,333,163,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
7,ㅡSHINㅡ,루이스,근거리딜러,추격자의 발걸음,전장의 학살자,거인 사냥꾼,S 아이스 에이지,유니크,S 윈터 크루제이드,유니크,S 프로즌 시티,유니크,S 아이스 토퀴즈,유니크,S 절대영도,유니크,E 크리스탈 크랙,유니크,아이스링크-결정,레어,샤드 플레이크,레어,아이스 스파클링,레어,아이스 엑셀,레어,아이스 이펙트,레어,솔리드 스위퍼,레어,하드스킨 버스터,레어,S 프로즌 하트,유니크,DB드라이 데브,레어,프로즌 브레이크,유니크,False,60,8,4,15,36919,22171,99,124,19300,3200,30,28,919,50,310,161,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
8,똥밟았네,멜빈,원거리딜러,전장의 열기,전장의 학살자,준비된 한 발,EP비헤이비어 암,레어,EP백워드 리즈닝,레어,라운드티 S5,커먼,듀얼벨트 C형,커먼,슬림핏 102,커먼,모노슈즈 M1,커먼,부트업 프라임,레어,EMP 프라임,레어,메카닉 스파클링,레어,아드레날린II,레어,메카닉 이펙트,레어,바이벤 스테민II,언커먼,바이퍼 임팩트,언커먼,E 황혼의 태엽,유니크,바이퍼 프라임,레어,머시너리 매니아,유니크,False,65,12,2,16,39982,12637,152,135,19100,1700,7,33,919,189,466,56,0,faa5ff5973d5d78ffa9f9741409d5f8d2242f33e0789ec...
9,폴로랄프로렌,플로리안,탱커,불굴의 의지,재생의 숨결,최후의 저항,S 빼앗은 증표,유니크,E 나르시시즘,유니크,E 영원한 소년,유니크,S 프레데터,유니크,S 에고 디펜스,유니크,S 스포일드 차일드,유니크,늪지대 이

In [ ]:
#
playerId = get_playerid('용1현')
matchids = get_matches(playerId)
antdpData = road_antdpData()
now_matcs = []
for v in matchids:
    if v not in antdpData:
        now_matcs.append(v)

In [ ]:
antdpData_save(antdpData,now_matcs)

In [ ]:
antdpData

In [35]:
game['map']['name']

'그랑플람 아시아 지부'

In [ ]:
for i in range(0,16):
        print(game['players'][2]['items'][i]['itemName']+'  '+game['players'][2]['items'][i]['rarityName'])

In [ ]:
basic = pd.DataFrame(columns=['nickname','characterName','position','attribute1','attribute2','attribute3','hand',
                              'head','chest','waist','leg','foot','accessories1','accessories2','recovery','attack','accelerator','defense','special','neck','accessories3','accessories4','random','level','killCount','deathCount',
                              'assistCount','attackPoint','damagePoint','battlePoint','sightPoint','getCoin','spendConsumablesCoin','sentinelKillCount','demolisherKillCount','playTime','minLifeTime','maxLifeTime','responseTime','team','matchId'])

In [ ]:
basic.info()

In [ ]:
basic

In [ ]:
basic2

In [ ]:
basic
#1은 이긴팀 0은 진팀이다.

In [ ]:
basic2.info()

In [ ]:
for i in range(0,10):
    if '3c3ed41120978a011ed2813fc2e53f13' == game['players'][i]['playerId']:
        

In [ ]:
for i in range(0,10):
    print(game['players'][i]['playerId']

In [ ]:
game

In [ ]:
playerId = get_playerid('똥밟았네')
matchids = get_matches(playerId)
win = []
lose = []
for matchid in matchids:
    winer_team,loser_team = win_and_lose(matchid)
    time.sleep(0.2)
    win.append(winer_team)
    lose.append(loser_team)
test = pd.DataFrame(win,columns=['c1','c2','c3','c4','c5'])
test['win']=1
test2 = pd.DataFrame(lose,columns=['c1','c2','c3','c4','c5'])
test2['win']=0
test3 = pd.concat([test,test2],join='inner')
test3 = test3.reset_index(drop=True, inplace=False)

test3 = test3.replace('로라스',1).replace('휴톤',2).replace('루이스',3).replace('타라',4).replace('트리비아',5).replace('카인',6).replace('레나',7).replace('드렉슬러',8).replace('도일',9).replace('토마스',10).replace('나이오비',11).replace('시바',12).replace('웨슬리',13).replace('스텔라',14).replace('앨리셔',15).replace('클레어',16).replace('다이무스',17).replace('이글',18).replace('마를렌',19).replace('샬럿',20).replace('윌라드',21).replace('레이튼',22).replace('미쉘',23).replace('린',24).replace('빅터',25).replace('카를로스',26).replace('호타루',27).replace('트릭시',28).replace('히카르도',29).replace('까미유',30).replace('자네트',31).replace('피터',32).replace('아이작',33).replace('레베카',34).replace('엘리',35).replace('마틴',36).replace('브루스',37).replace('미아',38).replace('드니스',39).replace('제레온',40).replace('루시',41).replace('티엔',42).replace('하랑',43).replace('제이',44).replace('벨져',45).replace('리첼',46).replace('리사',47).replace('릭',48).replace('제키엘',49).replace('탄야',50).replace('캐럴',51).replace('라이샌더',52).replace('디아나',53).replace('클리브',54).replace('헬레나',55).replace('에바',56).replace('론',57).replace('레오노르',58).replace('시드니',59).replace('테이',60).replace('티모시',61).replace('엘프리데',62).replace('티샤',63).replace('카로슈',64).replace('라이언',65).replace('파수꾼 A',66).replace('에밀리',67).replace('플로리안',68).replace('케니스',69).replace('이사벨',70).replace('헤나투',71).replace('루드빅',72).replace('멜빈',73)
#라이샌더뒤로 루드빅을 넣어서 하나씩 밀어야함(임시)
test3 = test3.fillna(0)
# train = pd.read_csv('train.csv')
# train.drop(['Unnamed: 0'], axis = 1, inplace = True)
# train = pd.concat([train,test3],join='inner')
# train = train.reset_index(drop=True, inplace=False)
# train.to_csv('train.csv')

In [ ]:
now_matcs

In [ ]:
now_matcs

In [ ]:
train

In [ ]:
X_train = train.drop('win',axis=1)
y_train = train.win

In [ ]:
X_test = test3.drop('win',axis=1)
y_test = test3.win

In [ ]:
import lightgbm as lgb

lgb_clf = lgb.LGBMClassifier(max_depth = 10,min_child_samples=60,subsample=0.8)
lgb_clf.fit(X_train, y_train)
y_pred = lgb_clf.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
X_train = pd.concat([X_train,X_test],join='inner')
X_train = X_train.reset_index(drop=True, inplace=False)
y_train = pd.concat([y_train,y_test],join='inner')
y_train = y_train.reset_index(drop=True, inplace=False)
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')

In [ ]:
y_pred

In [ ]:
y_train